# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import ipaddress as ip
import pickle
import requests, bs4
from datetime import datetime
import math

In [3]:
pd.options.display.max_columns = 50

# Importing Data and EDA

In [4]:
dataset = pd.read_csv('data_cleaned.csv')

In [5]:
dataset.head()

,url,label
0,http://diaryofagameaddict.com,1
1,http://espdesign.com.au,1
2,http://iamagameaddict.com,1
3,http://kalantzis.net,1
4,http://slightlyoffcenter.net,1


In [6]:
len(dataset)

420464

In [7]:
dataset['label'].value_counts()

0    344821
1     75643
Name: label, dtype: int64

# Feature Generartion

##### 01.  Long URL to Hide the Suspicious Part
Phishers can use long URL to hide the doubtful part in the address bar. For example: 
http://federmacedoadv.com.br/3f/aze/ab51e2e319e51502f416dbe46b773a5e/?cmd=_home&amp;dispatch=11004d58f5b74f8dc1e7c2e8dd4105e811004d58f5b74f8dc1e7c2e8dd4105e8@phishing.website.html

In [8]:
length = lambda x : len(x)

##### 02. Using the IP Address
If an IP address is used as an alternative of the domain name in the URL, such as “http://125.98.3.123/fake.html”, users can be sure that someone is trying to steal their personal information. Sometimes, the IP address is even transformed into hexadecimal code as shown in the following link “http://0x58.0xCC.0xCA.0x62/2/paypal.ca/index.html”. 

If the domain part has an IP address -> Phishing

Otherwise -> Legitimate

In [9]:
def isIp(x):
    try:
        if ip.ip_address(x):
            return 1
    except:
        return 0

##### 03. URL’s having “@” Symbol
Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol. 

In [10]:
countAt = lambda x : x.count('@')

##### 04. Redirecting using “//”
The existence of “//” within the URL path means that the user will be redirected to another website. An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. We examin the location where the “//” appears. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.


In [11]:
countDoubleSlash = lambda x : x.count('//')

##### 05. Adding Prefix or Suffix Separated by (-) to the Domain
The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage. For example http://www.Confirme-paypal.com/.


In [12]:
countHyphen = lambda x : x.count('-')

##### 06. Sub Domain and Multi Sub Domains
Let us assume we have the following link: http://www.hud.ac.uk/students/. A domain name might include the country-code top-level domains (ccTLD), which in our example is “uk”. The “ac” part is shorthand for “academic”, the combined “ac.uk” is called a second-level domain (SLD) and “hud” is the actual name of the domain. To produce a rule for extracting this feature, we firstly have to omit the (www.) from the URL which is in fact a sub domain in itself. Then, we have to remove the (ccTLD) if it exists. Finally, we count the remaining dots. If the number of dots is greater than one, then the URL is classified as “Suspicious” since it has one sub domain. However, if the dots are greater than two, it is classified as “Phishing” since it will have multiple sub domains. Otherwise, if the URL has no sub domains, we will assign “Legitimate” to the feature. 


In [13]:
countDots = lambda x: x.count('.')

##### 07. num of delimeters:
[';','_','?','=','&']

In [14]:
def countDelimeters(x):
    count = 0
    for delim in [';','_','?','=','&']:
        count += x.count(delim)
    return count

##### 08. subdirectory count

In [15]:
countSubDirectory = lambda x : x.count('/')

##### 09. query count

In [16]:
def countQueries(x):
    if not x:
        return 0
    else:
        return len(x.split('&'))

##### 10. Alexa Global Rank

In [17]:
def alexaGlobalRank(x):
    try:
        res = requests.get('https://www.alexa.com/siteinfo/'+x)
        if res.status_code == requests.codes.ok:
            websitesoup = bs4.BeautifulSoup(res.text)
            elems = websitesoup.select('strong')
            elem = elems[0]
            if elem.getText() == "We don't have enough data to rank this website.":
                return -1
            else:
                elem = elems[6]
                return int(elem.getText().strip())
        else:
            return -1
    except:
        return -1

##### Domain Registration Length
Based on the fact that a phishing website lives for a short period of time, we believe that trustworthy domains are regularly paid for several years in advance. In our dataset, we find that the longest fraudulent domains have been used for one year only. 


In [18]:
to_datetime = lambda x : datetime(int(x.getText()[:4]), int(x.getText()[5:7]), int(x.getText()[8:]))

In [19]:
def domainRegistrationLength(x):
    try:
        res = requests.get('https://www.whois.com/whois/'+x)
        if res.status_code == requests.codes.ok:
            websitesoup = bs4.BeautifulSoup(res.text)
            elems = websitesoup.select('.df-value')
            regOn, expOn = elems[2:4]
            regOn = to_datetime(regOn)
            expOn = to_datetime(expOn)
            duration = (expOn - regOn).total_seconds()//31556926
            return int(duration)
        else:
            return -1
    except:
        return -1

##### Using URL Shortening Services “TinyURL”
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL. For example, the URL “http://portal.hud.ac.uk/” can be shortened to “bit.ly/19DXSk4”.


##### URL of Anchor
An anchor is an element defined by the 'a' tag. This feature is treated exactly as “Request URL”.

In [20]:
def generateFeatures(x):
    features = []
    features.append(length(x))
    features.append(isIp(x))
    features.append(countAt(x))
    features.append(countDoubleSlash(x))
    features.append(countHyphen(x))
    features.append(countDots(x))
    features.append(countDelimeters(x))
    features.append(countSubDirectory(x))
    features.append(countQueries(x))
    features.append(alexaGlobalRank(x))
    return features

In [21]:
def generateTokens(x):
    allTokens = []
    tokenSlash = x.split('/')
    for ts in tokenSlash:
        tokensHyphen = ts.split('-')
        for th in tokensHyphen:
            tokensDot = th.split('.')
            allTokens += tokensDot
    return list(set(allTokens))

In [22]:
featureset = pd.DataFrame(dataset['url'].copy())

In [23]:
featureset.head()

,url
0,http://diaryofagameaddict.com
1,http://espdesign.com.au
2,http://iamagameaddict.com
3,http://kalantzis.net
4,http://slightlyoffcenter.net


In [24]:
featureset['length'] = featureset['url'].apply(length)

In [25]:
featureset['isIp'] = featureset['url'].apply(isIp)

In [26]:
featureset['countAt'] = featureset['url'].apply(countAt)

In [27]:
featureset['countDoubleSlash'] = featureset['url'].apply(countDoubleSlash)

In [28]:
featureset['countHyphen'] = featureset['url'].apply(countHyphen)

In [29]:
featureset['countDots'] = featureset['url'].apply(countDots)

In [30]:
featureset['countDelimeters'] = featureset['url'].apply(countDelimeters)

In [31]:
featureset['countSubDirectory'] = featureset['url'].apply(countSubDirectory)

In [32]:
featureset['countQueries'] = featureset['url'].apply(countQueries)

In [ ]:
featureset.head()

In [ ]:
X = featureset.drop(['url'], axis=1)
y = dataset['label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y)

# Machine Learning

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR2 = LogisticRegression()
LR2.fit(X_train, y_train)
print(LR2.score(X_eval, y_eval))

final_LR2 = LogisticRegression()
final_LR2.fit(X, y)

pickle.dump(final_LR2, open('LogisticRegression2.pickle', 'wb'))

## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF2 = RandomForestClassifier()
RF2.fit(X_train, y_train)
print(RF2.score(X_eval, y_eval))

final_RF2 = RandomForestClassifier()
final_RF2.fit(X, y)

pickle.dump(final_RF2, open('RandomForestClassifier2.pickle', 'wb'))

# Testing

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test.head()

In [ ]:
len(test)

In [ ]:
X_test = pd.DataFrame(columns=featureset.columns.drop('url'))

In [ ]:
X_test.head()

In [ ]:
for i in range(len(test)):
    features = generateFeatures(test['URL'].loc[i])    
    X_test.loc[i] = features

In [ ]:
X_test.head()

In [ ]:
y_test = test['Label']

## Logistic Regression

In [ ]:
y_pred = final_LR2.predict(X_test)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

In [ ]:
y_pred = final_RF2.predict(X_test)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))